In [50]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras

In [51]:
def training(len_input, embed_space_size, embed_action_size):
    '''define layer'''
    input_space = keras.Input(shape=len_input[0], name='Space_input')
    input_action = keras.Input(shape=len_input[0], name='Action_input')
    input_other = keras.Input(shape=len_input, name='Other_input')

    space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')
    
    input_concat_embedding = layers.Concatenate(name='Input_merging', axis=2)
    layer_dense = layers.Dense(units=3, activation='softmax')

    '''define forward'''
    inputs = [input_space, input_action, input_other]
    embeded_space = space_embedding(input_space)
    embeded_action = action_embeding(input_action)

    embeded_input = input_concat_embedding([embeded_space, embeded_action, input_other])
    output_prob = layer_dense(embeded_input)

    model = keras.Model(inputs=inputs, outputs=output_prob, name='classifier')

    return model

---

prepare data

In [52]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

In [53]:
train_input = train_df[['Team', 'No.', 'Space', 'Action']]
train_label = train_df[['Erros', 'Score']]

In [54]:
train_space = train_input[['Space']]
train_action = train_input[['Action']]
train_others = train_input[['Team', 'No.']]

train_space = train_space.astype({'Space': 'str'})
train_action = train_action.astype({'Action': 'str'})
train_others = train_others.astype({'No.': 'str'})

In [55]:
# train_space_dummy = pd.get_dummies(train_space)
# train_action_dummy = pd.get_dummies(train_action)
train_others_dummy = pd.get_dummies(train_others)

In [56]:
train_label.loc[:, 'Nothing'] = 0.0
train_label.loc[(train_label['Erros'] == 0) & (train_label['Score'] == 0), 'Nothing'] = 1.0

c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
c:\Users\YiChen\anaconda3\envs\SportsScience\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be se

In [57]:
print(train_space.shape)
print(train_action.shape)
print(train_others_dummy.shape)

print(train_label.shape)

(900, 1)
(900, 1)
(900, 24)
(900, 3)


In [58]:
# Avoid tensorflow use full memory
physical_devices = tf.config.experimental.list_physical_devices('CPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [66]:
embed_space_size = 14
embed_action_size = 8
model = training(
    train_others_dummy.shape, 
    embed_space_size, 
    embed_action_size
    )
    
model.summary()

Model: "classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Space_input (InputLayer)        [(None, 900)]        0                                            
__________________________________________________________________________________________________
Action_input (InputLayer)       [(None, 900)]        0                                            
__________________________________________________________________________________________________
Space_embedding (Embedding)     (None, 900, 4)       56          Space_input[0][0]                
__________________________________________________________________________________________________
Action_embedding (Embedding)    (None, 900, 4)       32          Action_input[0][0]               
_________________________________________________________________________________________

In [60]:
regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['AUC']
epochs = 20
callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [61]:
train_space_tensor = tf.convert_to_tensor(train_space)
train_action_tensor = tf.convert_to_tensor(train_action)
train_others_tensor = tf.convert_to_tensor(train_others_dummy)
train_label_tensor = tf.convert_to_tensor(train_label)

train_space_tensor = tf.expand_dims(train_space_tensor, 0)
train_action_tensor = tf.expand_dims(train_action_tensor, 0)
train_others_tensor = tf.expand_dims(train_others_tensor, 0)
train_label_tensor = tf.expand_dims(train_label_tensor, 0)


train_x = [train_space_tensor, train_action_tensor, train_others_tensor]

In [65]:
pred = model.fit(train_x, train_label_tensor, epochs=epochs, callbacks=[callbacks])

ValueError: Calling `Model.fit` in graph mode is not supported when the `Model` instance was constructed with eager mode enabled. Please construct your `Model` instance in graph mode or call `Model.fit` with eager mode enabled.